In [4]:
import cv2
import dlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde

# Load pre-trained models for face and landmarks
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

def process_video(video_path, output_heatmap="heatmap.png"):
    # Open the video
    cap = cv2.VideoCapture(video_path)
    gaze_points = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert to grayscale for processing
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Detect faces
        faces = face_detector(gray_frame)
        for face in faces:
            # Get facial landmarks
            landmarks = landmark_predictor(gray_frame, face)
            left_eye = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)]
            right_eye = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)]

            # Compute the center of both eyes
            left_eye_center = np.mean(left_eye, axis=0).astype(int)
            right_eye_center = np.mean(right_eye, axis=0).astype(int)

            # Append gaze points
            gaze_points.append(left_eye_center)
            gaze_points.append(right_eye_center)

            # Draw eyes on the frame (for visualization, optional)
            cv2.circle(frame, tuple(left_eye_center), 3, (255, 0, 0), -1)
            cv2.circle(frame, tuple(right_eye_center), 3, (255, 0, 0), -1)

        # Display the frame (optional, for debugging)
        cv2.imshow("Processing Video", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Generate Heatmap
    generate_heatmap(gaze_points, output_heatmap)

def generate_heatmap(gaze_points, output_file):
    # Convert gaze points to x and y coordinates
    x_coords = [point[0] for point in gaze_points]
    y_coords = [point[1] for point in gaze_points]

    # Create a heatmap using seaborn
    plt.figure(figsize=(10, 6))
    sns.kdeplot(x=x_coords, y=y_coords, cmap="Reds", shade=True, bw_adjust=0.5)
    plt.title("Eye Gaze Heatmap")

    plt.gca().invert_yaxis()  # Invert y-axis to match image coordinates
    plt.savefig(output_file)
    plt.show()

    print(f"Heatmap sa|ved as {output_file}")

# Example Usage
process_video("trial.mp4", output_heatmap="eye_gaze_heatmap.png")


RuntimeError: Unable to open shape_predictor_68_face_landmarks.dat